In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

import pylab as plt
%matplotlib inline

import seaborn as sns

## Get data

### SES data

In [6]:
pe = pd.read_csv('PovertyEstimates.csv', thousands=',').rename(columns={'FIPStxt': 'fips'})
pe['fips'] = pe['fips'].apply(lambda x: str(x).zfill(5))
pe = pe.set_index('fips')[['POVALL_2018', 'PCTPOVALL_2018', 'MEDHHINC_2018']]

### Rural data

In [7]:
df_rural = pd.read_csv('County_Rural_Lookup.csv', thousands=',', usecols=[0, 7])
df_rural = df_rural.rename(columns={'2015 GEOID': 'fips', '2010 Census \nPercent Rural': 'perc_rural_pop'})
df_rural = df_rural.set_index('fips').dropna()

### Demographic data

In [8]:
dm_raw = pd.read_csv('cc-est2019-alldata.csv', encoding='ISO-8859-1', dtype={'STATE': str, 'COUNTY': str})
dm_raw = dm_raw[dm_raw.YEAR==12]
dm_raw['fips'] = dm_raw.STATE + dm_raw.COUNTY
dm_raw = dm_raw.set_index('fips')
dm_raw.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
fips,,,,,,,,,,,,,,,,,,,,,
01001,50,01,001,Alabama,Autauga County,12,0,55869,27092,28777,...,778,687,89,93,40,27,15,19,16,11
01001,50,01,001,Alabama,Autauga County,12,1,3277,1713,1564,...,76,53,10,6,6,5,3,4,3,3
01001,50,01,001,Alabama,Autauga County,12,2,3465,1787,1678,...,83,59,2,10,8,2,2,0,1,1
01001,50,01,001,Alabama,Autauga County,12,3,3851,1977,1874,...,84,67,11,12,2,2,1,2,2,1
01001,50,01,001,Alabama,Autauga County,12,4,3659,1854,1805,...,55,68,7,6,4,5,0,4,3,0


In [9]:
dm_raw['minority'] = dm_raw.TOT_POP - (dm_raw.WA_MALE + dm_raw.WA_FEMALE)
dm_raw['black'] = dm_raw.BA_MALE + dm_raw.BA_FEMALE
dm_raw['hispanic'] = dm_raw.H_MALE + dm_raw.H_FEMALE
dm_all = dm_raw[dm_raw.AGEGRP == 0][['minority', 'black', 'hispanic', 'TOT_POP']]

In [10]:
dm_old = dm_raw[['AGEGRP', 'TOT_POP']][dm_raw.AGEGRP >= 14].reset_index().groupby('fips').sum()
dm_old = dm_old.rename(columns={'TOT_POP': '65yrs'}).drop(['AGEGRP'], axis=1)

In [11]:
dm = dm_old.join(dm_all)

In [12]:
columns = dm.columns
for c in ['65yrs', 'minority', 'black', 'hispanic']:
    pc = 'perc_' + c
    dm[pc] = dm[c] / dm.TOT_POP

### Sanity check for demographic dataframe (dm)
passed

In [13]:
summation = dm.sum(axis=0)
perc_black = summation['black'] / summation['TOT_POP']
print(f'black percentage = {100 * perc_black:.1f}%')
perc_hispanic = summation['hispanic'] / summation['TOT_POP']
print(f'hispanic percentage = {100 * perc_hispanic:.1f}%')
perc_minority = summation['minority'] / summation['TOT_POP']
print(f'minority percentage = {100 * perc_minority:.1f}%')
print(f"population over 65yrs = {summation['65yrs']}")

black percentage = 13.4%
hispanic percentage = 18.5%
minority percentage = 23.7%
population over 65yrs = 54058263.0


### Covid data

In [22]:
df_covid_raw = pd.read_csv('time_series_covid19_confirmed_US_2020-10-17.csv').dropna()
df_covid_raw['FIPS'] = df_covid_raw['FIPS'].apply(lambda x: str(int(x)).zfill(5))
df_covid_raw = df_covid_raw.rename(columns={'FIPS': 'fips'}).set_index('fips')
df_covid_raw = df_covid_raw.drop([
    'UID', 'iso2', 'iso3', 'code3', 
    'Admin2', 'Province_State', 'Country_Region', 
    'Lat', 'Long_', 'Combined_Key'], axis=1)

covid_chosen = ['4/15/20', '6/15/20', '7/15/20', '9/15/20', df_covid_raw.columns[-1]]

df_covid_chosen = df_covid_raw[covid_chosen]
df_covid_period = df_covid_raw[['4/15/20']].copy()

for i in range(len(covid_chosen))[::-1][:-1]:
    col = covid_chosen[i]
    col_prev = covid_chosen[i - 1]
    df_covid_period[col] = df_covid_chosen[col] - df_covid_chosen[col_prev]

df_covid_period = df_covid_period[covid_chosen]
df_covid_period['covid'] = df_covid_raw[df_covid_raw.columns[-1]]

In [34]:
df_covid = df_covid_period.join(dm[['TOT_POP']])
for col in df_covid_period.columns:
    new_col = col + 'pc'
    df_covid[new_col] = df_covid[col] / df_covid['TOT_POP']
df_covid.drop(['TOT_POP'], axis=1, inplace=True)
df_covid.head()

,4/15/20,6/15/20,7/15/20,9/15/20,10/17/20,covid,4/15/20pc,6/15/20pc,7/15/20pc,9/15/20pc,10/17/20pc,covidpc
fips,,,,,,,,,,,,
01001,24,340.0,379.0,720.0,520.0,1983.0,0.000430,0.006086,0.006784,0.012887,0.009307,0.035494
01003,91,277.0,1131.0,3313.0,1538.0,6350.0,0.000408,0.001241,0.005066,0.014841,0.006890,0.028445
01005,12,225.0,189.0,203.0,348.0,977.0,0.000486,0.009114,0.007656,0.008223,0.014097,0.039577
01007,18,90.0,130.0,342.0,195.0,775.0,0.000804,0.004019,0.005805,0.015272,0.008708,0.034607
01009,17,97.0,253.0,772.0,668.0,1807.0,0.000294,0.001677,0.004375,0.013350,0.011552,0.031249


### Risk data
For Ishanu:
1. The `A_B` column of the risk dataframe `rf` means llk of disease model `B` generating disease time series `A`;
2. Now risk is defined to be `2 / (llk Staph generating Staph + llk Strep generating Strep)`;
3. Now risk_flu is defined to be `1 / llk of flu model generating flu sequence`;
4. Feel free to try other formulae for risk.

In [14]:
rf = pd.read_csv('county_pop_risk_covid.csv', dtype={'county': str})
rf = rf.rename(columns={'county': 'fips'}).set_index('fips')
rf['risk'] = 2 / (rf['Staphylococcus_Staphylococcus'] + rf['Streptococcal_Streptococcal'])

### Combine demographic, SES, rural, risk, and covid dataframe
For Ishanu: df here has a column of population over 65yrs

In [37]:
risk_cols = ['risk', 'risk_flu']
df = dm.join(pe, how='inner')\
    .join(rf[risk_cols], how='inner')\
    .join(df_rural, how='inner')\
    .join(df_covid, how='inner')\
    .rename(columns={
        'POVALL_2018': 'poverty',
        'PCTPOVALL_2018': 'perc_poverty', 
        'MEDHHINC_2018': 'income', 
        'TOT_POP': 'population'})
df.columns

Index(['65yrs', 'minority', 'black', 'hispanic', 'population', 'perc_65yrs',
       'perc_minority', 'perc_black', 'perc_hispanic', 'poverty',
       'perc_poverty', 'income', 'risk', 'risk_flu', 'perc_rural_pop',
       '4/15/20', '6/15/20', '7/15/20', '9/15/20', '10/17/20', 'covid',
       '4/15/20pc', '6/15/20pc', '7/15/20pc', '9/15/20pc', '10/17/20pc',
       'covidpc'],
      dtype='object')

## Preprocess for regression

In [13]:
df_regr = df[[
    'perc_65yrs',
    'perc_minority',
    'perc_black', 
    'perc_hispanic', 
    'perc_poverty',
    'income', 
    'population',
    'risk',
    'risk_flu', 
    'covid', 
    'covidpc'
]]
# df_regr['risk_flu*population'] = df_regr['risk_flu'] * df_regr['population']

In [14]:
covariates = [
    'population', 
    'perc_65yrs', 
    'perc_minority', 
    'perc_black', 
    'perc_hispanic', 
    'perc_poverty', 
    'income',
    'risk', 
    'risk_flu',
    # 'risk_flu*population'
]

### Z-ify the covariates and `covidpc`. 
But don't z-ify `covid` because we are going to fit a Poisson

In [15]:
df_regr_z = df[['covid', 'covidpc']].copy()
for c in covariates + ['covidpc']:
    mean, std = df_regr[c].mean(), df_regr[c].std()
    df_regr_z[c] = (df_regr[c] - mean) / std

df_regr_z.head()

,covid,covidpc,population,perc_65yrs,perc_minority,perc_black,perc_hispanic,perc_poverty,income,risk,risk_flu
fips,,,,,,,,,,,
01001,1839,0.834474,-0.149053,-0.794110,0.512704,0.741331,-0.489701,-0.225017,0.480936,0.758899,0.815284
01003,6116,0.464835,0.349320,0.257407,-0.180112,-0.042335,-0.365710,-0.883379,0.353735,1.060673,1.170239
01005,923,1.133801,-0.241908,-0.012918,2.192827,2.685013,-0.379630,2.589482,-1.333023,-0.049015,-0.009922
01007,691,0.696529,-0.248733,-0.647764,0.481482,0.821242,-0.504694,1.091708,-0.483902,0.035787,0.084791
01009,1665,0.557914,-0.143225,-0.220996,-0.699751,-0.532157,-0.011633,-0.323771,-0.167862,0.282300,0.338405


## Determine Variance Inflation Faction
Since we are using Generalized Linear Model, we may not need to mention VIF

In [16]:
y, X = dmatrices(
    'covidpc~' + '+'.join(covariates), 
    df_regr_z, 
    return_type='dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.round(1)

,VIF Factor,features
0,1.0,Intercept
1,1.6,population
2,1.6,perc_65yrs
3,4.5,perc_minority
4,4.0,perc_black
5,1.2,perc_hispanic
6,4.5,perc_poverty
7,4.1,income
8,294.1,risk
9,295.4,risk_flu


## GLM Models

### With both risk

In [17]:
formula_covidpc = f"covidpc~{'+'.join(covariates)}" 
formula_covid = f"covid~{'+'.join(covariates)}" 
print(formula_covid)
print(formula_covidpc)

covid~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+income+risk+risk_flu
covidpc~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+income+risk+risk_flu


In [18]:
model_covidpc = smf.glm(
    formula=formula_covidpc,
    data=df_regr_z,
    family=sm.families.Gaussian(sm.families.links.log())
).fit()
print(model_covidpc.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                covidpc   No. Observations:                 3086
Model:                            GLM   Df Residuals:                     3076
Model Family:                Gaussian   Df Model:                            9
Link Function:                    log   Scale:                         0.83637
Method:                          IRLS   Log-Likelihood:                -4098.1
Date:                Fri, 23 Oct 2020   Deviance:                       2572.7
Time:                        00:51:07   Pearson chi2:                 2.57e+03
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -2.5589      0.106    -24.158

In [19]:
model_covid = smf.glm(
    formula=formula_covid,
    data=df_regr_z,
    family=sm.families.Poisson(sm.families.links.log())
).fit()
print(model_covid.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  covid   No. Observations:                 3086
Model:                            GLM   Df Residuals:                     3076
Model Family:                 Poisson   Df Model:                            9
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5511e+06
Date:                Fri, 23 Oct 2020   Deviance:                   3.0778e+06
Time:                        00:51:17   Pearson chi2:                 3.86e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.7162      0.001   9748.568

### Without risk

In [20]:
formula_covid = 'covid~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+income'
formula_covidpc = 'covidpc~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+income'

In [21]:
model_covidpc = smf.glm(
    formula=formula_covidpc,
    data=df_regr_z,
    family=sm.families.Gaussian(sm.families.links.log())
).fit()
print(model_covidpc.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                covidpc   No. Observations:                 3094
Model:                            GLM   Df Residuals:                     3086
Model Family:                Gaussian   Df Model:                            7
Link Function:                    log   Scale:                         0.83175
Method:                          IRLS   Log-Likelihood:                -4101.2
Date:                Fri, 23 Oct 2020   Deviance:                       2566.8
Time:                        00:51:30   Pearson chi2:                 2.57e+03
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -2.3569      0.128    -18.409

In [22]:
model_covid = smf.glm(
    formula=formula_covid,
    data=df_regr_z,
    family=sm.families.Poisson(sm.families.links.log())
).fit()
print(model_covid.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  covid   No. Observations:                 3094
Model:                            GLM   Df Residuals:                     3086
Model Family:                 Poisson   Df Model:                            7
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.1485e+06
Date:                Fri, 23 Oct 2020   Deviance:                   8.2723e+06
Time:                        00:51:31   Pearson chi2:                 1.13e+07
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.0960      0.001   1.25e+04

### With only risk_flu

In [23]:
formula_covid = 'covid~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+risk_flu'
formula_covidpc = 'covidpc~population+perc_65yrs+perc_minority+perc_black+perc_hispanic+perc_poverty+risk_flu'

In [24]:
model_covidpc = smf.glm(
    formula=formula_covidpc,
    data=df_regr_z,
    family=sm.families.Gaussian(sm.families.links.log())
).fit()
print(model_covidpc.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                covidpc   No. Observations:                 3086
Model:                            GLM   Df Residuals:                     3078
Model Family:                Gaussian   Df Model:                            7
Link Function:                    log   Scale:                         0.84104
Method:                          IRLS   Log-Likelihood:                -4107.7
Date:                Fri, 23 Oct 2020   Deviance:                       2588.7
Time:                        00:51:57   Pearson chi2:                 2.59e+03
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -2.6103      0.128    -20.429

In [25]:
model_covid = smf.glm(
    formula=formula_covid,
    data=df_regr_z,
    family=sm.families.Poisson(sm.families.links.log())
).fit()
print(model_covid.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  covid   No. Observations:                 3086
Model:                            GLM   Df Residuals:                     3078
Model Family:                 Poisson   Df Model:                            7
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5683e+06
Date:                Fri, 23 Oct 2020   Deviance:                   3.1121e+06
Time:                        00:51:58   Pearson chi2:                 3.91e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         6.7143      0.001   9745.462